In [2]:
import awkward as ak
from coffea import processor
from coffea.nanoevents.methods import candidate
import hist
import pandas as pd
import numpy as np
import pathlib
import shutil
import os
import matplotlib.pyplot as plt
import mplhep as hep
import warnings
warnings.filterwarnings('ignore')

In [3]:
class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass

    def process(self, events):
        dataset = events.metadata['dataset']
        info = ak.zip(
            {
                "runNum": events.runNum,
                "lumiNum": events.lumiNum,
                "evtNum": events.eventNum,
                "nevt": events.eventCounter,
                "ishlt":events.isHLT_PPZeroBias,
            }
        )
        
        ca4 = ak.zip(
            {
                'ncsc':events.nca4CSCcluster,
                'eta':events.ca4CSCclusterEta,
                'phi':events.ca4CSCclusterPhi,
                'x':events.ca4CSCclusterX,        
                'y':events.ca4CSCclusterY,
                'z':events.ca4CSCclusterZ,                
                'size':events.ca4CSCclusterSize, # need for HLT numerator
                'time':events.ca4CSCclusterTime,
                'timeSpread':events.ca4CSCclusterTimeSpread,            
                "nME11_12": events.ca4CSCclusterME11_12,
                'time':events.ca4CSCclusterTime,            
                "nStation10": events.ca4CSCclusterNstation10,
                "avgStation10": events.ca4CSCclusterAvgStation10,            
            }
        )
        
        elctHMT = ak.zip(
            {
                'bits': events.elctHMT_bits,
                'WireNHits': events.elctHMT_WireNHits,
                'sr': events.elctHMT_sr,
            }
        )
        
        ### Preselection based on HLT_ZeroBias_v7
        presel = info.ishlt
        ca4 = ca4[presel]
        elctHMT = elctHMT[presel]
        
        ## Threshold array
        # arr1 = np.arange(25, 76, 5)
        # arr2 = np.arange(10, 25)
        # arr = np.concatenate((arr2, arr1), axis=None)
        arr = np.arange(10, 31)
        
        names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']

        sta_axis = hist.axis.StrCategory(names, growth=True)
        cls_axis = hist.axis.Regular(50, 10, 1000, name="clusterSize")
        thr_axis = hist.axis.IntCategory(arr)

        h_denom = hist.Hist(sta_axis, cls_axis, label="Denom")
        h_numer = hist.Hist(sta_axis, thr_axis, cls_axis, label="Numer")
        
        ca4.R = (ca4.x**2+ ca4.y**2)**0.5
        
        ## Define station based on 
        ME11 = (ca4.R>100)&(ca4.R<275) &(abs(ca4.z)>580)&(abs(ca4.z)<632) 
        ME12 = (ca4.R>275)&(ca4.R<465) &(abs(ca4.z)>668)&(abs(ca4.z)<724)
        ME13 = (ca4.R>505)&(ca4.R<700) &(abs(ca4.z)>668)&(abs(ca4.z)<724)

        ME21 = (ca4.R>139)&(ca4.R<345) &(abs(ca4.z)>789)&(abs(ca4.z)<850)
        ME22 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>791)&(abs(ca4.z)<850)

        ME31 = (ca4.R>160)&(ca4.R<345) &(abs(ca4.z)>915)&(abs(ca4.z)<970)
        ME32 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>911)&(abs(ca4.z)<970)

        ME41 = (ca4.R>178)&(ca4.R<345) &(abs(ca4.z)>1002)&(abs(ca4.z)<1063)
        ME42 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>1002)&(abs(ca4.z)<1063)
        
        for i, region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
            sel = (ak.num(ca4, axis=1) > 0) & ak.all(region & (ca4.nStation10==1) & (ca4.ncsc==1), axis=1)
            #masked_ca4_size = ak.mask(ca4.size, sel)                
            denom = ak.fill_none(ak.max(ca4[sel].size, axis=1), -1)     
            h_denom.fill(names[i], denom)
            
            
            # masked_elctHMT = ak.mask(elctHMT.WireNHits, sel)
            # tmp = ak.mask(denom, ak.any(elctHMT.bits > 1, axis=1))
        
            for thres in arr:
                numer = ak.fill_none(denom[ak.any(elctHMT[sel].WireNHits>thres,axis=1)],-1) 
                # tmp2 = ak.mask(tmp, ak.any(masked_elctHMT > thres, axis=1))
                # numer = ak.fill_none(tmp2, -1)
                h_numer.fill(names[i], thres, numer)
                  
        return {
            'h_denom': h_denom,
            'h_numer': h_numer,
        }

    def postprocess(self, accumulator):
        return accumulator

In [9]:
import time
from coffea.nanoevents import BaseSchema

tstart = time.time()

futures_run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=12),
    schema=BaseSchema,
    maxchunks=50000,
)

fileset = {
    'run2022c': open("ppZeroBias_Run2022C_hltinclude.txt").read().split("\n"),
    'run2022e': open("ppZeroBias_Run2022E_hltinclude.txt").read().split("\n")
}

# fileset = {
#     'test': [
#         './plots_487.root',
#     ],
# }

output = futures_run(
    fileset,
    treename="simpleCSCshowerFilter/hmt",
    processor_instance=MyProcessor(),
)

elapsed = time.time() - tstart

print(output)

Output()

Output()

{'h_denom': Hist(
  StrCategory(['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42'], growth=True, label='Axis 0'),
  Regular(50, 10, 1000, name='clusterSize'),
  storage=Double()) # Sum: 690671.0 (690673.0 with flow), 'h_numer': Hist(
  StrCategory(['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42'], growth=True, label='Axis 0'),
  IntCategory([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], label='Axis 1'),
  Regular(50, 10, 1000, name='clusterSize'),
  storage=Double()) # Sum: 788199.0 (788240.0 with flow)}


In [11]:
output['h_numer']['ME21', 0, :]

Hist(Regular(50, 10, 1000, name='clusterSize'), storage=Double()) # Sum: 21923.0

In [10]:
import pickle
filename = 'histograms_hltpath_zb_v7.pickle'
outfile = open(filename, 'wb')
pickle.dump(output, outfile)
outfile.close()

In [ ]:
# ### Before hlt path included

# class MyProcessor(processor.ProcessorABC):
#     def __init__(self):
#         pass

#     def process(self, events):
#         dataset = events.metadata['dataset']
#         info = ak.zip(
#             {
#                 "runNum": events.runNum,
#                 "lumiNum": events.lumiNum,
#                 "evtNum": events.eventNum,
#                 "nevt": events.eventCounter,
#                 "ishlt":events.isHLT_PPZeroBias,
#             }
#         )
        
#         ca4 = ak.zip(
#             {
#                 'ncsc':events.nca4CSCcluster,
#                 'eta':events.ca4CSCclusterEta,
#                 'phi':events.ca4CSCclusterPhi,
#                 'x':events.ca4CSCclusterX,        
#                 'y':events.ca4CSCclusterY,
#                 'z':events.ca4CSCclusterZ,                
#                 'size':events.ca4CSCclusterSize, # need for HLT numerator
#                 'time':events.ca4CSCclusterTime,
#                 'timeSpread':events.ca4CSCclusterTimeSpread,            
#                 "nME11_12": events.ca4CSCclusterME11_12,
#                 'time':events.ca4CSCclusterTime,            
#                 "nStation10": events.ca4CSCclusterNstation10,
#                 "avgStation10": events.ca4CSCclusterAvgStation10,            
#             }
#         )
        
#         elctHMT = ak.zip(
#             {
#                 'bits': events.elctHMT_bits,
#                 'WireNHits': events.elctHMT_WireNHits,
#                 'sr': events.elctHMT_sr,
#             }
#         )
        
#         ## Threshold array
#         arr1 = np.arange(25, 76, 5)
#         arr2 = np.arange(10, 25)
#         arr = np.concatenate((arr2, arr1), axis=None)
        
#         names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']

#         sta_axis = hist.axis.StrCategory(names, growth=True)
#         cls_axis = hist.axis.Regular(50, 10, 1000, name="clusterSize")
#         thr_axis = hist.axis.IntCategory(arr)

#         h_denom = hist.Hist(sta_axis, cls_axis, label="Denom")
#         h_numer = hist.Hist(sta_axis, thr_axis, cls_axis, label="Numer")
        
#         ca4.R = (ca4.x**2+ ca4.y**2)**0.5
        
#         ## Define station based on 
#         ME11 = (ca4.R>100)&(ca4.R<275) &(abs(ca4.z)>580)&(abs(ca4.z)<632) 
#         ME12 = (ca4.R>275)&(ca4.R<465) &(abs(ca4.z)>668)&(abs(ca4.z)<724)
#         ME13 = (ca4.R>505)&(ca4.R<700) &(abs(ca4.z)>668)&(abs(ca4.z)<724)

#         ME21 = (ca4.R>139)&(ca4.R<345) &(abs(ca4.z)>789)&(abs(ca4.z)<850)
#         ME22 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>791)&(abs(ca4.z)<850)

#         ME31 = (ca4.R>160)&(ca4.R<345) &(abs(ca4.z)>915)&(abs(ca4.z)<970)
#         ME32 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>911)&(abs(ca4.z)<970)

#         ME41 = (ca4.R>178)&(ca4.R<345) &(abs(ca4.z)>1002)&(abs(ca4.z)<1063)
#         ME42 = (ca4.R>357)&(ca4.R<700) &(abs(ca4.z)>1002)&(abs(ca4.z)<1063)
        
# #         def dump_pandas(pddf: pd.DataFrame, fname: str):       
# #             dirname = str(dataset)
# #             if not os.path.exists(dirname):
# #                 pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
            
# #             local_file = (os.path.abspath(os.path.join(".", dirname, fname)))
# #             pddf.to_parquet(local_file)
        
# #         df_denom = pd.DataFrame()    
# #         df_numer = pd.DataFrame()
        
#         for i, region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
#             sel = (ak.num(ca4, axis=1) > 0) & ak.all(region & (ca4.nStation10==1) & (ca4.ncsc==1), axis=1)
#             masked_ca4_size = ak.mask(ca4.size, sel)                
#             denom = ak.fill_none(ak.max(masked_ca4_size, axis=1), -1)     
                                          
#             h_denom.fill(names[i], denom)
            
#             masked_elctHMT = ak.mask(elctHMT.WireNHits, sel)
#             tmp = ak.mask(denom, ak.any(elctHMT.bits > 1, axis=1))
        
#             for thres in arr:
#                 tmp2 = ak.mask(tmp, ak.any(masked_elctHMT > thres, axis=1))
#                 numer = ak.fill_none(tmp2, -1)
#                 h_numer.fill(names[i], thres, numer)
                  
#         return {
#             'h_denom': h_denom,
#             'h_numer': h_numer,
#         }

#     def postprocess(self, accumulator):
#         return accumulator

In [ ]:
# import time
# from coffea.nanoevents import BaseSchema

# tstart = time.time()

# futures_run = processor.Runner(
#     executor = processor.FuturesExecutor(compression=None, workers=12),
#     schema=BaseSchema,
#     maxchunks=50000,
# )

# # fileset = {
# #     'run2022c': open("ppZeroBias_Run2022C_hltinclude.txt").read().split("\n"),
# #     'run2022e': open("ppZeroBias_Run2022E_hltinclude.txt").read().split("\n")
# # }

# fileset = {
#     'test': [
#         './plots_487.root',
#     ],
# }

# output = futures_run(
#     fileset,
#     treename="simpleCSCshowerFilter/hmt",
#     processor_instance=MyProcessor(),
# )

# elapsed = time.time() - tstart

# print(output)

In [ ]:
import pickle
filename = 'histograms_v2.pickle'
outfile = open(filename, 'wb')
pickle.dump(output, outfile)
outfile.close()

In [ ]:
output['h_numer']['ME11', 0, :]

In [ ]:
output['h_denom']['ME11', :]

In [ ]:
main_ax_artists, sublot_ax_arists = output['h_numer']['ME11', 10, :].plot_ratio(
    output['h_denom']['ME11', :],
    rp_ylabel=r"Ratio",
    rp_num_label="Numer",
    rp_denom_label="Denom",
    rp_uncert_draw_type="line",  # line or bar
)

In [ ]:
#         df = pd.DataFrame()
        
#         totalevt = ak.sum(info.nevt, axis=0)
#         #print(totalevt)
        
#         # remove empty sub-array 
#         for thres in arr:
#             csc_threshold = ak.any(elctHMT.WireNHits > thres, axis=1)
#             tmp = elctHMT[(ak.num(elctHMT, axis=1) > 0) & (ak.any(elctHMT.bits >= 2, axis=1)) & (csc_threshold)]

#             ME11_sr = (tmp.sr==8) | (tmp.sr==9)
#             ME12_sr = (tmp.sr==7) | (tmp.sr==10)
#             ME13_sr = (tmp.sr==6) | (tmp.sr==11)

#             ME21_sr = (tmp.sr==5) | (tmp.sr==12)
#             ME22_sr = (tmp.sr==4) | (tmp.sr==13)

#             ME31_sr = (tmp.sr==3) | (tmp.sr==14)
#             ME32_sr = (tmp.sr==2) | (tmp.sr==15)

#             ME41_sr = (tmp.sr==1) | (tmp.sr==16)
#             ME42_sr = (tmp.sr==0) | (tmp.sr==17)
        
#             names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']
#             cnts = []
#             for i, region in enumerate([ME11_sr,ME12_sr,ME13_sr,ME21_sr,ME22_sr,ME31_sr,ME32_sr,ME41_sr,ME42_sr]):
#                 nevt = ak.count_nonzero(ak.any(region, axis=1))
#                 cnts.append(nevt)
            
#             df[str(thres)] = cnts
        
#         def dump_pandas(pddf: pd.DataFrame, fname: str):       
#             dirname = str(dataset)
#             if not os.path.exists(dirname):
#                 pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
            
#             local_file = (os.path.abspath(os.path.join(".", dirname, fname)))
#             pddf.to_parquet(local_file)
            
#         fname = (events.behavior["__events_factory__"]._partition_key.split('/')[0] + "_nevt.parquet")
#         dump_pandas(df, fname)
        
#         ratio_df = df/totalevt
#         fname = (events.behavior["__events_factory__"]._partition_key.split('/')[0] + "_ratio.parquet")
#         dump_pandas(ratio_df, fname)

In [ ]:
pd.read_parquet('test/8836f4fe-d39e-11ed-8024-a70f030abeef_denom.parquet')

In [ ]:
pd.read_parquet('test/8836f4fe-d39e-11ed-8024-a70f030abeef_numer.parquet')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 6))
#output['test']['wirehits'].plot1d(ax=ax, overlay='station')
#ax.set_xscale("log")
#ax.legend(title="Dimuon charge")

In [ ]:
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
ak.behavior.update(candidate.behavior)

def getLZDF(f,treename ="hmt",nEvents=-1):
    events_raw = uproot.open(f)[treename]
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    events = events[start:stop]
    if treename=="hmt":
        return pack(events)
    else:
        return events
        

def pack(events):
    events.info=ak.zip({
        "runNum": events.runNum,
        "lumiNum": events.lumiNum,
        "evtNum": events.eventNum,
        }
    )
    events.cls=ak.zip({
        'eta':events.ca4CSCclusterEta,
        'phi':events.ca4CSCclusterPhi,
        'x':events.ca4CSCclusterX,        
        'y':events.ca4CSCclusterY,
        'z':events.ca4CSCclusterZ,                
        'size':events.ca4CSCclusterSize,
        'time':events.ca4CSCclusterTime,
        'timeSpread':events.ca4CSCclusterTimeSpread,            
        "nME11_12": events.ca4CSCclusterME11_12,
        'time':events.ca4CSCclusterTime,            
        "nStation10": events.ca4CSCclusterNstation10,
        "avgStation10": events.ca4CSCclusterAvgStation10,            
        })
    return events

In [ ]:
Nevents = -1
samples = {}
for f in glob('../tmp/*.root'):
    name = str(f).split('/')[2].split('.')[0]
    hmt = getLZDF(f,'simpleCSCshowerFilter/hmt',Nevents)
    samples[name] = hmt[hmt.isHLT_HIZeroBias]

In [ ]:
arr_threshold1 = range(10, 75, 5)
arr_threshold2 = range(10, 25)

In [ ]:
## Count the number of events depending on the threshold of CSC per each station
## Let's use pandas

df = pd.DataFrame()

for sample in samples.keys():
    #if not sample == 'HITestRaw1': break
    
    tmp_df = pd.DataFrame()
    
    print('Which sample?', sample)
    hmt =  samples[sample]
    elctHMT = ak.zip({k.replace("elctHMT_",""):getattr(hmt,k) for k in hmt.fields if k.startswith("elctHMT_")})
    
    emul_sel = elctHMT.bits>=2
    elctHMT = elctHMT[emul_sel] # first selection for nominal and tight selections
    
    for i, threshold in enumerate(arr_threshold2):
        cnts = []
        
        emul_thres = elctHMT.WireNHits > threshold
        tmp = elctHMT[emul_thres]
        
        ME11_sr = (tmp.sr==8) | (tmp.sr==9)
        ME12_sr = (tmp.sr==7) | (tmp.sr==10)
        ME13_sr = (tmp.sr==6) | (tmp.sr==11)

        ME21_sr = (tmp.sr==5) | (tmp.sr==12)
        ME22_sr = (tmp.sr==4) | (tmp.sr==13)

        ME31_sr = (tmp.sr==3) | (tmp.sr==14)
        ME32_sr = (tmp.sr==2) | (tmp.sr==15)

        ME41_sr = (tmp.sr==1) | (tmp.sr==16)
        ME42_sr = (tmp.sr==0) | (tmp.sr==17)
    
        for i, region in enumerate([ME11_sr,ME12_sr,ME13_sr,ME21_sr,ME22_sr,ME31_sr,ME32_sr,ME41_sr,ME42_sr]):
            cnts.append(ak.count_nonzero(region))
        
        tmp_df[str(threshold)] = cnts
        
    ### Let's make a final one
    df = df.add(tmp_df, fill_value=0)

In [ ]:
df.to_csv('events_threshold10to24.csv', index=False)

In [ ]:
sumlist = df.sum(axis=0)
df.loc[9] = sumlist

In [ ]:
rate_df = df*1e-4*2500
rate_df

In [ ]:
# Add station name in the first position


df.insert(0, "station", names)
rate_df.insert(0, "station", names)

In [ ]:
rate_df

In [ ]:
names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42', 'total']
x = range(10, 75, 5)
plt.figure(figsize=(10,10))
plt.grid()
plt.yscale('log')
plt.ylabel('# of emulated HMT fired events', fontsize=15)
plt.xlabel('CSC threshold', fontsize=15)
plt.xticks(x)

for i in range(0, 10):
    evts = df.loc[i, :].values.flatten()
    plt.plot(x, evts, label=names[i], marker='o')

plt.legend()
plt.savefig('number_of_events_per_station_by_thresholds.png')

In [ ]:
names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42', 'total']
x = arr_threshold2
plt.figure(figsize=(10,10))
plt.grid()
plt.yscale('log')
plt.ylabel('Emulated HMT rates (Hz)', fontsize=15)
plt.xlabel('CSC threshold', fontsize=15)
plt.xticks(x)

#rate_df = rate_df.drop(columns=['station'])
for i in range(0, 10):
    evts = rate_df.loc[i, :].values.flatten()
    plt.plot(x, evts, label=names[i], marker='o')

plt.legend()
plt.savefig('trigger_rate_per_station_by_thresholds.png')

In [ ]:
cls = ak.zip(
    {k.replace("ca4CSCcluster",""):getattr(hmt,k) for k in hmt.fields if k.startswith("ca4CSCcluster")}
    ,with_name="PtEtaPhiMLorentzVector", 
    behavior=vector.behavior
    )

lctHMT = ak.zip({k.replace("lctHMT_",""):getattr(hmt,k) for k in hmt.fields if k.startswith("lctHMT_")})
elctHMT = ak.zip({k.replace("elctHMT_",""):getattr(hmt,k) for k in hmt.fields if k.startswith("elctHMT_")})

In [ ]:
names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']

## Consider nominal and tight selections
emul_sel = elctHMT.bits>=2
elctHMT = elctHMT[emul_sel] # first selection for nominal and tight selections

## Station selections
ME11_sr = (elctHMT.sr==8)|(elctHMT.sr==9)
ME12_sr = (elctHMT.sr==7)|(elctHMT.sr==10)
ME13_sr = (elctHMT.sr==6)|(elctHMT.sr==11)

ME21_sr = (elctHMT.sr==5)|(elctHMT.sr==12)
ME22_sr = (elctHMT.sr==4)|(elctHMT.sr==13)

ME31_sr = (elctHMT.sr==3)|(elctHMT.sr==14)
ME32_sr = (elctHMT.sr==2)|(elctHMT.sr==15)

ME41_sr = (elctHMT.sr==1)|(elctHMT.sr==16)
ME42_sr = (elctHMT.sr==0)|(elctHMT.sr==17)

In [ ]:
for i, region in enumerate([ME11_sr,ME12_sr,ME13_sr,ME21_sr,ME22_sr,ME31_sr,ME32_sr,ME41_sr,ME42_sr]):
    print(names[i], ak.count_nonzero(region))

In [ ]:
cls.R = (cls.X**2+ cls.Y**2)**0.5

ME11 = (cls.R>100)&(cls.R<275) &(abs(cls.Z)>580)&(abs(cls.Z)<632) 
ME12 = (cls.R>275)&(cls.R<465) &(abs(cls.Z)>668)&(abs(cls.Z)<724)
ME13 = (cls.R>505)&(cls.R<700) &(abs(cls.Z)>668)&(abs(cls.Z)<724)

ME21 = (cls.R>139)&(cls.R<345) &(abs(cls.Z)>789)&(abs(cls.Z)<850)
ME22 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>791)&(abs(cls.Z)<850)

ME31 = (cls.R>160)&(cls.R<345) &(abs(cls.Z)>915)&(abs(cls.Z)<970)
ME32 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>911)&(abs(cls.Z)<970)

ME41 = (cls.R>178)&(cls.R<345) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)
ME42 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)

In [ ]:
#thresholds = [140,140,14,56,28,55,26,62,27]
thresholds = [20,20,20,20,20,20,20,20,20]

for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.any(region&(cls.Nstation10==1),axis=1)
    denom = ak.fill_none(ak.max(cls[sel].Size,axis=1),-1)
    print(i, denom)
    numer = ak.fill_none(denom[ak.any(elctHMT[sel].WireNHits>thresholds[i],axis=1)],-1)
    print(numer)

In [ ]:
ak.count_nonzero(ME11)

In [ ]:
ak.count_nonzero(cls.Nstation10==1)

In [ ]:
cls.Nstation10==1

In [ ]:
test = (region & (cls.Nstation10==1))

In [ ]:
ak.count_nonzero(test)

In [ ]:
ak.any(test, axis=1)

In [ ]:
c = ak.any(test, axis=1)

# L1 efficiency v.s. station

In [ ]:
ME11 = (cls.R>100)&(cls.R<275) &(abs(cls.Z)>580)&(abs(cls.Z)<632) 
ME12 = (cls.R>275)&(cls.R<465) &(abs(cls.Z)>668)&(abs(cls.Z)<724)
ME13 = (cls.R>505)&(cls.R<700) &(abs(cls.Z)>668)&(abs(cls.Z)<724)

ME21 = (cls.R>139)&(cls.R<345) &(abs(cls.Z)>789)&(abs(cls.Z)<850)
ME22 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>791)&(abs(cls.Z)<850)

ME31 = (cls.R>160)&(cls.R<345) &(abs(cls.Z)>915)&(abs(cls.Z)<970)
ME32 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>911)&(abs(cls.Z)<970)

ME41 = (cls.R>178)&(cls.R<345) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)
ME42 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)

plt.style.use(hep.style.CMS) 
fig, axs = plt.subplots(3,3, figsize=(24,18))
axs = axs.flatten()

def plotEff(numer,denom,ax):
    h= hist.Hist("Events",hist.Cat("sample","sample"),hist.Bin("ClusterSize", "ClusterSize", 50, 10, 1000))        
    h.fill(sample="numer",ClusterSize=numer)
    h.fill(sample="denom",ClusterSize=denom)

    hist.plotratio(num=h.integrate("sample",'numer'),
                   denom=h.integrate("sample",'denom'),
                    xerr=True,
                   error_opts={"linestyle":'none',},
                   ax=ax,clear=False
                  )
    return ax


for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.all(region&(cls.Nstation10==1),axis=1) 
    denom = ak.fill_none(ak.max(cls[sel].Size,axis=1),-1)
    numer = ak.fill_none(denom[hmt[sel].passL1==1],-1)    
#     plotEff(numer,denom,axs[i])
    
#thresholds       = [140,14,28,26]
#tight_thresholds = [140,18,32,34]

#thresholds       = [140,140,14,56,28,55,26,62,27]
# two_loose        = [140,20  ,8,28 ,9,26 ,8,31,13]
thresholds = [20,20,20,20,20,20,20,20,20]
#two_loose        = np.ceil(np.array([140,20  ,8,28 ,9,26 ,8,31,13])*0.8)

#for i,ax_i in enumerate([1,2,4,6]):
for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.all(region&(cls.Nstation10==1),axis=1) 
    denom = ak.fill_none(ak.max(cls[sel].Size,axis=1),-1)
    numer = ak.fill_none(denom[ak.any(elctHMT[sel].WireNHits>thresholds[i],axis=1)],-1)    
    #numer_twoLoose = ak.fill_none(denom[ak.any(elctHMT[sel].WireNHits>two_loose[i],axis=1)],-1)            
#     numer_tight = ak.fill_none(denom[ak.any(elctHMT[sel].WireNHits>tight_thresholds[i],axis=1)],-1)        
#     plotEff(numer,denom,axs[ax_i])
#     plotEff(numer_tight,denom,axs[ax_i])    
    plotEff(numer,denom,axs[i])
    #plotEff(numer_twoLoose,denom,axs[i])    

    
labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]
runNumber = "run 362321"
for i,ax in enumerate(axs):
    ax.set_ylabel("L1 Efficiency")
    ax.set_xlabel("Max. ClusterSize(CA4)")
    if i in [1,2,4,6,8]:
        ax.vlines(200,0,2,color='black',label="HLT threshold",lw=3)    
    else:
        ax.vlines(500,0,2,color='black',label="HLT threshold",lw=3)
#     if i in [1,2,4,6]:
#         ax.legend(["%s"%labels[i],"nominal","Two-loose"],title=runNumber)    
#     else:
#         ax.legend(["%s"%labels[i]],title=runNumber)            
    #ax.legend(["HLT threshold",'nominal','Two-loose thres.'],title="%s"%labels[i])
    ax.legend(["HLT threshold",'nominal'],title="%s"%labels[i])
    ax.set_ylim(0,2)
    plt.tight_layout()

In [ ]:
fig.savefig('l1_efficiency.png')